<a href="https://colab.research.google.com/github/anjali-rgpt/argue-better/blob/main/Result_Analysis_224N_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=cae61a0f7dcfc8d3ff6b2aa4daa34d450b5b976af56d5b861480235d41758ba4
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
data = pd.read_csv('/content/augmented_predictions_all.csv')

In [3]:
# sum(df['augmented_predictions'].str.contains("\u00a0"))

In [4]:
# sum(df['augmented_predictions'].str.contains("\xa0"))

In [5]:
# sum(df['augmented_predictions'].str.contains("\n"))

In [6]:
%pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
%pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/bleurt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 11.5 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456781 sha2

In [7]:
sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [111]:
from statistics import  mode
from sklearn.metrics.pairwise import cosine_similarity

In [9]:

query_embeddings = sentence_embedding_model.encode(list(data['discourse_text']))


In [10]:

ground_truth = []
count = 0
for index, row in data.iterrows():
  candidates = row[['predictions', 'context_predictions', 'augmented_predictions']].values
  if len(np.unique(candidates)) > 1:
    ground_truth.append(mode(candidates))
  else:
    q_embedding = [query_embeddings[count]]
    other_embeddings = sentence_embedding_model.encode(candidates)
    similarity = cosine_similarity(q_embedding, other_embeddings)
    ground_truth.append(candidates[np.argmax(similarity[0])])
  
  count += 1
    
  # print(ground_truth)
  

In [11]:
len(data)

27439

In [12]:
from bleurt import score

In [13]:
references = ground_truth
candidates1 = data['predictions']

The idea behind BLEURT is that the metric is pretrained on millions of synthetic sentences with perturbations and other manipulations that keep the original meaning relatively similar (as well as on other natural language generation objectives), and then it is fine-tuned on human ratings on the WMT Metrics dataset. 

In [14]:
scorer = score.BleurtScorer()

In [15]:
scores = scorer.score(references=references, candidates=candidates1)

In [16]:
np.argmin(scores)

4572

In [17]:
references[np.argmin(scores)]

"When I was a Seagoing Cowboy, it opened up the world to me. I saw that the world wasn't all sunshine and rainbows, like the little me thought. The world was full of unfairness and was my job, everybody's job, to help stop it. I stepped up to do my part by joining the Seagoing Cowboys. Do your part. Join the Seagoing Cowboys program today. Make the world a better place, one cow at a time. "

In [18]:
candidates1[np.argmin(scores)]

'You get to travel on seas! '

In [19]:
candidates2 = data['context_predictions']
candidates3 = data['augmented_predictions']

In [20]:
scores2 = scorer.score(references=references, candidates=candidates2)


In [21]:
print(np.argmin(scores2), candidates2[np.argmin(scores2)],"/", references[np.argmin(scores2)])
print(np.argmax(scores2), candidates2[np.argmax(scores2)], "/", references[np.argmax(scores2)])

18810 the cars aren't completely driverless,  / hackers 
15682 The problem is actually efficency.  / The problem is actually efficency. 


In [22]:
scores3 = scorer.score(references=references, candidates=candidates3)


In [23]:
print(scores3[np.argmin(scores3)], candidates2[np.argmin(scores3)], "<>", references[np.argmin(scores3)])
print(scores3[np.argmax(scores3)], candidates2[np.argmax(scores3)], "<>", references[np.argmax(scores3)])

-1.4608819484710693 It helps animals that might not make it otherwise live until they get wherever they are going, then someone can take care of them and make sure that they are okay.  <> When I was a Seagoing Cowboy, it opened up the world to me. I saw that the world wasn't all sunshine and rainbows, like the little me thought. The world was full of unfairness and was my job, everybody's job, to help stop it. I stepped up to do my part by joining the Seagoing Cowboys. Do your part. Join the Seagoing Cowboys program today. Make the world a better place, one cow at a time. 
0.9384633898735046 Although I understand why you would reconsider the policy I believe it could go either way.

Some students may take advantage of the policy and secretly use it in class.

Others might take embarrassing photos of somebody else and ruin their life.

The only way to stop this from happening is to ban cell phones for good, however that can also take away the privileges that some students deserve, so wh

In [24]:
print(np.mean(scores3))

-0.002646777868605402


# For fine-tuning on augmented dataset

In [25]:
df = pd.read_csv('/content/finetune_gpt2_example_aug_full.csv')

In [26]:
df.head()

,Unnamed: 0,input,pred,true
0,0,Or when somebody got hurt real bad and they ha...,If some of their closest friends have family...,"also, students would have to use a school phon..."
1,1,You might say I have to clean up poop gross.,As a poop-face it is obvious that you need y...,"Garbage on the grass, gum on the sidewalk that..."
2,2,I reconsider that are principal should let us ...,"First of all, if students have cell phones t...",I think that you should think of allowing stud...
3,3,Some parents may argue that distance learning ...,Many may disagree that taking away these ass...,Although many say that distance learning encou...
4,4,I believe we should keep the Electoral College...,The electoral college should be kept in the ...,We should keep the Electoral College for the v...


In [27]:
cands = list(df.pred)
refs = list(df.true)

In [28]:
scores_gpt2 = scorer.score(references=refs, candidates=cands)


In [29]:
print(scores_gpt2[np.argmin(scores_gpt2)], cands[np.argmin(scores_gpt2)], "<>", refs[np.argmin(scores_gpt2)])
print(scores_gpt2[np.argmax(scores_gpt2)], cands[np.argmax(scores_gpt2)], "<>", refs[np.argmax(scores_gpt2)])

-1.3579964637756348   First off, the cameras that we have today are more advanced than the ones we had.  <> Even if the cameras used to take these pictures weren't of the highest quality, said cameras would have been able to at least catch some photos of further signs of life. Things such as houses or buildings, for example. Now, if some one would like to argue that the aliens lived in caves or other natural formations of the sort, how would those people like to argue the fact that there are no visible creatures in any photos? Do they expect people to believe that every living thing on Mars decided to hide from a satelite they would not even be able to see from the surface? 
0.897011935710907   Next, the author gives possible ways to study Venus safely, and effciently.  <> Next, the author gives possible ways to study Venus safely, and effciently. 


In [30]:
np.mean(scores_gpt2)

-0.7502356443335029

In [31]:
cand_embeddings = sentence_embedding_model.encode(list(df.pred))
ref_embeddings = sentence_embedding_model.encode(list(df.true))

In [32]:
queries_gpt_aug = list(df['input'])

In [33]:
x = cosine_similarity(cand_embeddings, ref_embeddings)

In [34]:
similarities = []
for i in range(len(cand_embeddings)):
  similarities.append(cosine_similarity([cand_embeddings[i]], [ref_embeddings[i]]))

In [35]:
df.pred[10]

'  The use of technology to read the emotional expressions of the students in a classroom is unbelievably valuable.  '

In [36]:
df.true[10]

'The use of technology to read the emotional expressions of the students in a classroom is unbelievably valuable.  '

In [37]:
max(similarities)

array([[1.0000001]], dtype=float32)

Example-based GPT2 finetuning

In [38]:
gpt_example = pd.read_csv('/content/full_example.csv')

In [39]:
gpt_example.head()

,Unnamed: 0,input,pred,true
0,0,students will not get on,,students won't be able to get home.
1,1,I don't think the students would get benefits ...,\nWhen attending education from home by way of...,"Clearly, students will benefit from being able..."
2,2,The author mentions how Venus is most alike to...,\n,The author claims that the idea of inhabiting ...
3,3,Dear mr. senator i think that we should keep t...,\nI feel like keeping the Electoral College wo...,Dear Senator I am not in favor of keeping the ...
4,4,A mother of two who is also a media trainer na...,\nPeople find their transportation with a love...,"""All of our development since World War II has..."


In [40]:
gpt_example.fillna("\n", inplace = True)

In [41]:
refs_ex = list(gpt_example.true)
cands_ex = list(gpt_example.pred)
print(gpt_example.pred[gpt_example.pred.map(type)==float]) 

Series([], Name: pred, dtype: object)


In [42]:
queries_ex = list(gpt_example['input'])

In [43]:
# gpt_example.pred.value_counts()

In [44]:
scores_gpt2_ex = scorer.score(references=refs_ex, candidates=cands_ex)



In [45]:
print(scores_gpt2_ex[np.argmin(scores_gpt2_ex)], cands_ex[np.argmin(scores_gpt2_ex)], "<>", refs_ex[np.argmin(scores_gpt2_ex)])
print(scores_gpt2_ex[np.argmax(scores_gpt2_ex)], cands_ex[np.argmax(scores_gpt2_ex)], "<>", refs_ex[np.argmax(scores_gpt2_ex)])

-1.4866538047790527 iced <> Having more points of view on a topic is always a good thing 
0.8661123514175415   " Furthermore the electoral college avoids the problem of elections in which no candidate recieves a majority of the votes cast.  <> Furthermore, The electoral college avoids the problem of elections in which no candidate recieves a majority of the votes cast. 


In [46]:
np.mean(scores_gpt2_ex)

-0.8054954238371416

# Fastrag t5

In [47]:
ft5 = pd.read_csv('/content/res_t5.csv')
ft5.shape

(500, 4)

In [48]:
truth_t5 = []
indices = []
for index, row in ft5.iterrows():
  # print(q)
  try:
    corresponding_gt = data.loc[data['discourse_text'] == row['query']].index.values[0]
    # print(corresponding_gt)
    truth_t5.append(ground_truth[corresponding_gt])
    indices.append(index)
  except:
    pass

In [49]:
len(truth_t5)

427

In [50]:
refs_t5 = truth_t5
cands_t5 = list(ft5.loc[indices, 'res'])

In [51]:
queries_t5 = list(ft5.loc[indices, 'query'])

In [52]:
scores_t5_ex = scorer.score(references=refs_t5, candidates=cands_t5)


In [53]:
print(scores_t5_ex[np.argmin(scores_t5_ex)], cands_t5[np.argmin(scores_t5_ex)], "<>", refs_t5[np.argmin(scores_t5_ex)])
print(scores_t5_ex[np.argmax(scores_t5_ex)], cands_t5[np.argmax(scores_t5_ex)], "<>", refs_t5[np.argmax(scores_t5_ex)])

-1.264596939086914 If you like animals and travling this is the job for you! You can have so much adventure you would'nt even know what to do with it! I hope that all of these reasons convinced you to become a seagoing cowboy <> You get to travel on seas! 
0.8471724987030029 Venus is a challenge to get to with its hot surface, with its thick atmosphere, and atmospheric pressure. But scientists and astronomers are facinated by Venus, because they believe it had oceans and could have supported various forms of life. <> Venus is a challenge to get to with its hot surface, with its thick atmosphere, and atmospheric pressure. But scientists and astronomers are facinated by Venus, because they believe it had oceans and could have supported various forms of life. 


In [54]:
np.mean(scores_t5_ex)

-0.6246257539690239

## Final model - ColBERT PLAID

In [55]:
colbertplaid = pd.read_csv('/content/res.csv')

In [56]:
colbertplaid.head()

,Unnamed: 0,idx,query,res
0,0,0,I also would know that it's not a face of the ...,"It looked like an Egyptian pharoah, but turned..."
1,1,1,"Finally, Despite the chanlleges and dangers th...",Students have the tendency to not to things if...
2,2,2,It does not let us the people choose. The Elec...,Many argue that without the Electoral Collage ...
3,3,3,"if we adopt the popular vote system, what woul...",Although not one vote will decide an election ...
4,4,4,In conclusion if we could possibly bulid somet...,"Well, first of all, The atmospere of Venus has..."


In [57]:
truth_cp = []
indices = []
for index, row in colbertplaid.iterrows():
  # print(q)
  try:
    corresponding_gt = data.loc[data['discourse_text'] == row['query']].index.values[0]
    # print(corresponding_gt)
    truth_cp.append(ground_truth[corresponding_gt])
    indices.append(index)
  except:
    pass

In [58]:
refs_cp = truth_cp
cands_cp = list(colbertplaid.loc[indices, 'res'])
scores_cp_ex = scorer.score(references=refs_cp, candidates=cands_cp)


In [59]:
print(scores_cp_ex[np.argmin(scores_cp_ex)], cands_cp[np.argmin(scores_cp_ex)], "<>", refs_cp[np.argmin(scores_cp_ex)])
print(scores_cp_ex[np.argmax(scores_cp_ex)], cands_cp[np.argmax(scores_cp_ex)], "<>", refs_cp[np.argmax(scores_cp_ex)])

-1.4343461990356445 decisions are rethought  <> The last reason why I think you should choose Policy 1 is because students need to take a break from working constantly and make plans with friends 
0.9087077975273132 you might receive better ideas,  <> you might receive better ideas, 


In [60]:
queries_cp = list(colbertplaid.loc[indices, 'query'])

In [61]:
np.mean(scores_cp_ex)

-0.6027957232062655

In [62]:
def average_cosine_similarity(queries, references):
  similarities = []
  cand_embeddings = sentence_embedding_model.encode(list(queries))
  ref_embeddings = sentence_embedding_model.encode(list(references))
  for i in range(len(cand_embeddings)):
    similarities.append(cosine_similarity([cand_embeddings[i]], [ref_embeddings[i]]))
  return np.mean(similarities)

In [63]:
print("Average cosine similarity for retriever examples baseline:", average_cosine_similarity(references, candidates1))
print("Average cosine similarity for retriever context baseline:", average_cosine_similarity(references, candidates2))
print("Average cosine similarity for retriever augmented baseline:", average_cosine_similarity(references, candidates3))
print("Average cosine similarity for colbert examples baseline:", average_cosine_similarity(refs_cp, cands_cp))
print("Average cosine similarity for t5 examples baseline:", average_cosine_similarity(refs_t5, cands_t5))
print("Average cosine similarity for fine-tuned gpt2 examples:", average_cosine_similarity(refs_ex, cands_ex))
print("Average cosine similarity for fine-tuned gpt2 augmented context:", average_cosine_similarity(refs, cands))


# The above checks for how similar the candidates are to the reference or ground truth

Average cosine similarity for retriever examples baseline: 0.9493475
Average cosine similarity for retriever context baseline: 0.56818295
Average cosine similarity for retriever augmented baseline: 0.79040325
Average cosine similarity for colbert examples baseline: 0.5642023
Average cosine similarity for t5 examples baseline: 0.5639642
Average cosine similarity for fine-tuned gpt2 examples: 0.38985255
Average cosine similarity for fine-tuned gpt2 augmented context: 0.49214935


In [64]:
print("Average cosine similarity for retriever examples baseline:", average_cosine_similarity(data['discourse_text'], candidates1))
print("Average cosine similarity for retriever context baseline:", average_cosine_similarity(data['discourse_text'], candidates2))
print("Average cosine similarity for retriever augmented baseline:", average_cosine_similarity(data['discourse_text'], candidates3))
print("Average cosine similarity for colbert examples baseline:", average_cosine_similarity(queries_cp, cands_cp))
print("Average cosine similarity for t5 examples baseline:", average_cosine_similarity(queries_t5, cands_t5))
print("Average cosine similarity for fine-tuned gpt2 examples:", average_cosine_similarity(queries_ex, cands_ex))
print("Average cosine similarity for fine-tuned gpt2 augmented context:", average_cosine_similarity(queries_gpt_aug, cands))

# The above checks how similar the candidate is to the query in retaining meaning


Average cosine similarity for retriever examples baseline: 0.7070896
Average cosine similarity for retriever context baseline: 0.46088055
Average cosine similarity for retriever augmented baseline: 0.6567297
Average cosine similarity for colbert examples baseline: 0.52281314
Average cosine similarity for t5 examples baseline: 0.52713007
Average cosine similarity for fine-tuned gpt2 examples: 0.37014592
Average cosine similarity for fine-tuned gpt2 augmented context: 0.46640778


In [65]:
def average_sentence_length(candidates):
  # print("X")
  # print(np.min([len(x.split()) for x in candidates]))
  return np.mean([len(x.split()) for x in candidates])

In [66]:
print("Average length for retriever examples baseline:", average_sentence_length(candidates1))
print("Average length for retriever context baseline:", average_sentence_length(candidates2))
print("Average length for retriever augmented baseline:", average_sentence_length(candidates3))
print("Average length for colbert examples baseline:", average_sentence_length(cands_cp))
print("Average length for t5 examples baseline:", average_sentence_length(cands_t5))
print("Average length for fine-tuned gpt2 examples:", average_sentence_length(cands_ex))
print("Average length for fine-tuned gpt2 augmented context:", average_sentence_length(cands))
print("Average length for references:", average_sentence_length(ground_truth))

Average length for retriever examples baseline: 45.75720689529502
Average length for retriever context baseline: 50.35577098290754
Average length for retriever augmented baseline: 48.17066948503954
Average length for colbert examples baseline: 91.07962529274005
Average length for t5 examples baseline: 52.81733021077283
Average length for fine-tuned gpt2 examples: 41.658181818181816
Average length for fine-tuned gpt2 augmented context: 57.80882352941177
Average length for references: 46.022231130872115


## BERTScore

In [67]:
!pip install evaluate
from evaluate import load

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.3 MB/s eta 0:00:00


In [68]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00


In [69]:
from evaluate import load

bertscore = load("bertscore")

In [70]:
def average_bertscore(references, candidates):
  scores = bertscore.compute(predictions=candidates, references=references, model_type = 'distilbert-base-uncased')
  return {"precision" : np.mean(scores["precision"]), "recall" : np.mean(scores["recall"]), "f1" : np.mean(scores["f1"])}


In [71]:
del data

In [72]:
print("Average bertscore for retriever examples baseline:", average_bertscore(references, candidates1))
print("Average bertscore for retriever context baseline:", average_bertscore(references, candidates2))
print("Average bertscore for retriever augmented baseline:", average_bertscore(references, candidates3))
print("Average bertscore for colbert examples baseline:", average_bertscore(refs_cp, cands_cp))
print("Average bertscore for t5 examples baseline:", average_bertscore(refs_t5, cands_t5))
print("Average bertscore for fine-tuned gpt2 examples:", average_bertscore(refs_ex, cands_ex))
print("Average bertscore for fine-tuned gpt2 augmented context:", average_bertscore(refs, cands))




Average bertscore for retriever examples baseline: {'precision': 0.9683959246498532, 'recall': 0.9675833302091773, 'f1': 0.9678665732906426}
Average bertscore for retriever context baseline: {'precision': 0.7965680760164086, 'recall': 0.8018146837051784, 'f1': 0.7984516311588542}
Average bertscore for retriever augmented baseline: {'precision': 0.8698586512117641, 'recall': 0.8724561884975923, 'f1': 0.8706353435281232}
Average bertscore for colbert examples baseline: {'precision': 0.7627607570040701, 'recall': 0.7954797747542763, 'f1': 0.7779127247718793}
Average bertscore for t5 examples baseline: {'precision': 0.758221020743216, 'recall': 0.7792361677390909, 'f1': 0.7676790606500952}


Average bertscore for fine-tuned gpt2 examples: {'precision': 0.615597525509921, 'recall': 0.6196660533818331, 'f1': 0.6169729711792685}
Average bertscore for fine-tuned gpt2 augmented context: {'precision': 0.7373536869006998, 'recall': 0.7486140572411173, 'f1': 0.7421167567372322}


In [73]:
import gc
gc.collect()

0

# Is there a correlation between score and discourse element? Does the model do better on certain kinds of elements than others?

In [75]:
data = pd.read_csv('/content/augmented_predictions_all.csv')
  

In [126]:
def total_cosine_similarity(queries, references):
  similarities = []
  cand_embeddings = sentence_embedding_model.encode(list(queries))
  ref_embeddings = sentence_embedding_model.encode(list(references))
  for i in range(len(cand_embeddings)):
    similarities.append(cosine_similarity([cand_embeddings[i]], [ref_embeddings[i]]))
  return similarities

In [121]:
small_data = pd.DataFrame(data.loc[:, 'discourse_type'])

In [106]:
references = ground_truth
candidates1 = data['predictions']

In [107]:
len(ground_truth) == len(candidates1)

True

In [127]:
small_data['cosine_ex'] = total_cosine_similarity(ground_truth, candidates1)

In [130]:
small_data['cosine_aug'] = total_cosine_similarity(ground_truth, candidates3)

In [131]:
small_data.head()

,discourse_type,cosine_ex,cosine_aug
0,Lead,[[0.9999999]],[[0.9999999]]
1,Position,[[1.0]],[[1.0]]
2,Claim,[[1.0]],[[1.0]]
3,Evidence,[[1.0]],[[1.0]]
4,Counterclaim,[[0.9999998]],[[0.9999998]]


In [132]:
small_data.groupby('discourse_type').mean()

,cosine_ex,cosine_aug
discourse_type,,
Claim,[[0.9562852]],[[0.73260814]]
Concluding Statement,[[0.9692373]],[[0.7978327]]
Counterclaim,[[0.84232163]],[[0.9292264]]
Evidence,[[0.95896477]],[[0.7587474]]
Lead,[[0.9509168]],[[0.91645914]]
Position,[[0.9275279]],[[0.93926555]]
Rebuttal,[[0.9660714]],[[0.6723923]]


In [144]:
df.head()

,Unnamed: 0,input,pred,true
0,0,Or when somebody got hurt real bad and they ha...,If some of their closest friends have family...,"also, students would have to use a school phon..."
1,1,You might say I have to clean up poop gross.,As a poop-face it is obvious that you need y...,"Garbage on the grass, gum on the sidewalk that..."
2,2,I reconsider that are principal should let us ...,"First of all, if students have cell phones t...",I think that you should think of allowing stud...
3,3,Some parents may argue that distance learning ...,Many may disagree that taking away these ass...,Although many say that distance learning encou...
4,4,I believe we should keep the Electoral College...,The electoral college should be kept in the ...,We should keep the Electoral College for the v...


In [152]:
ground_truth_gpt_el = []
for example in gpt_example['input']:
  # print(data.loc[data['discourse_text'] == example.strip()]['discourse_type'])
  ground_truth_gpt_el.append(data.loc[data['discourse_text'] == example]['discourse_type'].values[0])

In [153]:
small_data_2 = pd.DataFrame(columns = ['discourse_element'])

In [154]:
small_data_2['discourse_type'] = ground_truth_gpt_el

In [155]:
small_data_2['gpt_sim'] = total_cosine_similarity(refs_ex, cands_ex)

In [156]:
small_data_2.groupby('discourse_type').mean()

,discourse_element,gpt_sim
discourse_type,,
Claim,NaN,[[0.33372506]]
Concluding Statement,NaN,[[0.48683771]]
Counterclaim,NaN,[[0.18513055]]
Evidence,NaN,[[0.39690346]]
Lead,NaN,[[0.4898848]]
Position,NaN,[[0.44046748]]
Rebuttal,NaN,[[0.34907556]]
